# Movie Recommendation System

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Data Collection and Pre-processing

In [2]:
movies_data = pd.read_csv("./data/movies.csv")
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [4]:
movies_data.shape

(4803, 24)

In [5]:
# selecting the relevant features for recommendation
selected_features = ["genres", "keywords", "tagline", "cast", "director"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
for feature in selected_features:
    print("Caracteristica:", feature)
    print("Valores nulos:", movies_data[movies_data[feature].isna()].shape)
    #print("Cadenas vacias:", movies_data[movies_data[feature] == ""].shape, "\n")

Caracteristica: genres
Valores nulos: (28, 24)
Caracteristica: keywords
Valores nulos: (412, 24)
Caracteristica: tagline
Valores nulos: (844, 24)
Caracteristica: cast
Valores nulos: (43, 24)
Caracteristica: director
Valores nulos: (30, 24)


In [7]:
# replacing the missing or null values on string features
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna("")

In [8]:
for feature in selected_features:
    print("Caracteristica:", feature)
    print("Valores nulos:", movies_data[movies_data[feature].isna()].shape)

Caracteristica: genres
Valores nulos: (0, 24)
Caracteristica: keywords
Valores nulos: (0, 24)
Caracteristica: tagline
Valores nulos: (0, 24)
Caracteristica: cast
Valores nulos: (0, 24)
Caracteristica: director
Valores nulos: (0, 24)


In [9]:
# combining all selected features
combined_features = movies_data[selected_features].agg(" ".join, axis=1)
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
# converting combined features (vector of text) to numerical values (numerical vector representation)
vectorizer = TfidfVectorizer(lowercase=True, min_df=1, stop_words='english')

In [11]:
feature_vector = vectorizer.fit_transform(combined_features)

In [12]:
print(feature_vector)

  (0, 2387)	0.17387762611499494
  (0, 7649)	0.11355691458889829
  (0, 12866)	0.19553338573938647
  (0, 10114)	0.16165931117025054
  (0, 8649)	0.22860674889222501
  (0, 14432)	0.1525185407602397
  (0, 16454)	0.19975784468313598
  (0, 13897)	0.2073363856597466
  (0, 13161)	0.21920124762693652
  (0, 17051)	0.20332801528845756
  (0, 16774)	0.23801225015751357
  (0, 13191)	0.15121581535902417
  (0, 11351)	0.2739303704041666
  (0, 16765)	0.12906888267705946
  (0, 4882)	0.2418630583829073
  (0, 14101)	0.21535043940154278
  (0, 3177)	0.25126855964819583
  (0, 16375)	0.1263324200909883
  (0, 14202)	0.3418956777020543
  (0, 5754)	0.16577485025092195
  (0, 3018)	0.22356693392633634
  (0, 3627)	0.21535043940154278
  (0, 5366)	0.10433355352725562
  (0, 13440)	0.10433355352725562
  (0, 5204)	0.11182749749508697
  :	:
  (4801, 13017)	0.29414364992580866
  (4801, 13672)	0.28404411818176767
  (4801, 17027)	0.29414364992580866
  (4801, 4774)	0.2518762905014918
  (4801, 397)	0.18067495553272095
  (4801, 

## Cosine similarity

La función `cosine_similarity` se utiliza para medir la similitud entre dos vectores, generalmente en el contexto de análisis de texto o aprendizaje automático. Su objetivo es calcular el coseno del ángulo entre los vectores, lo que proporciona una medida de cuán similares son en términos de dirección, independientemente de su magnitud.

### Rango de valores:
- **1**: Los vectores son idénticos (en cuanto a dirección).
- **0**: No tienen similitud (son ortogonales, es decir, no tienen ninguna relación).
- **-1**: Los vectores son opuestos.

### Fórmula matemática

La fórmula de la similitud del coseno es la siguiente:

$\text{cosine similarity} = \frac{A \cdot B}{\|A\| \|B\|}$

Donde:
- $A \cdot B$ es el producto punto entre los dos vectores.
- $\|A\|$ y $\|B\|$ son las normas (longitudes) de los vectores $A$ y $B$, respectivamente.

### Usos comunes
En la práctica, esta función se usa mucho en el procesamiento de lenguaje natural (PLN) para comparar la similitud entre:
- **Documentos**.
- **Palabras** representadas por vectores, como cuando se utiliza *TF-IDF* o *Word2Vec*.

La función nos devolverá una matriz cuadrada donde:
* Nro de filas: será el número de filas del vector que se está evaluando
* Nro de columnas: será el mismo número de filas ya que originalmente el vector tiene $n$ filas x 1 columna, y lo que hará es evaluar el valor de cada fila contra el resto de las filas. Entonces en la matriz nos quedaría, por ejemplo, en la primer fila el coseno del valor de a fila 0 por el de la fila 1, fila 2, fila 3, y asi sucesivamente para cada columna de cada fila de la matriz.

In [13]:
similarity = cosine_similarity(feature_vector)
similarity

array([[1.        , 0.04720973, 0.03891121, ..., 0.        , 0.        ,
        0.        ],
       [0.04720973, 1.        , 0.021542  , ..., 0.01267122, 0.        ,
        0.        ],
       [0.03891121, 0.021542  , 1.        , ..., 0.        , 0.05626942,
        0.        ],
       ...,
       [0.        , 0.01267122, 0.        , ..., 1.        , 0.        ,
        0.02730605],
       [0.        , 0.        , 0.05626942, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02730605, 0.        ,
        1.        ]])

In [14]:
similarity.shape

(4803, 4803)

In [18]:
movie_name = "iron man"

In [20]:
# creating a list with all the movies names
all_movies_names_list = movies_data["title"].tolist()
all_movies_names_list[0:5]

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter']

In [21]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, all_movies_names_list)
find_close_match

['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [22]:
close_match = find_close_match[0] if len(find_close_match) > 0 else "N/A"
close_match

'Iron Man'

In [29]:
# finding the index of the movie with that title
movie_index = movies_data[movies_data["title"] == close_match]["index"].values[0]
movie_index

68

La línea de código:

`list(enumerate(similarity[movie_index]))`

realiza lo siguiente paso a paso:

* `similarity[movie_index]`:
`similarity` probablemente es una estructura de datos como una lista o un array (como un array de NumPy o un DataFrame de Pandas) que contiene similitudes entre elementos (en este caso, películas).

* `movie_index` es un índice, es decir, un número que indica qué fila o entrada del objeto similarity estamos seleccionando. Esto devuelve una fila o un conjunto de valores que representa las similitudes de una película (indizada por movie_index) con respecto a otras películas.

* `enumerate(similarity[movie_index])`: `enumerate` es una función de Python que toma una iterable (en este caso, el valor de similarity[movie_index], que podría ser una lista o un array) y devuelve un objeto que produce tuplas en la forma (índice, valor), donde:
    - El índice es la posición del elemento dentro de la iterable (en este caso, la posición de la película que nos servirá para hallarla en el dataset original y ofrecer la sugerencia o recomendación al usuario).
    - El valor es el valor en esa posición, que en este caso podría ser la similitud entre la película en movie_index y otra película.

Entonces, `enumerate(similarity[movie_index])` genera un iterador que da tuplas (índice, similitud) para cada valor en la fila de similitudes de la película.

* `list(enumerate(similarity[movie_index]))`:
Finalmente, la función `list()` convierte ese iterador de tuplas en una lista. Esto te da una lista donde cada elemento es una tupla con el índice de la película y su similitud con la película en movie_index.

In [51]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[movie_index]))
similarity_score[0:5]

[(0, 0.034461890474908635),
 (1, 0.05844029953654316),
 (2, 0.01434810236042448),
 (3, 0.006617219324502558),
 (4, 0.036501767008759664)]

In [35]:
len(similarity_score)

4803

In [38]:
# sorting the movies based on their similarity score
# es decir, ordeno de mayor a menor los valores de similitud 
# para tener en primer lugar aquellos con similitud mas cercana a la pelicula que busco
sorted_similar_movies = sorted(similarity_score, key= lambda x: x[1], reverse=True)
sorted_similar_movies[0:5]

[(68, 1.0000000000000002),
 (79, 0.4434783547506395),
 (31, 0.3331597801593984),
 (7, 0.24824053933095896),
 (16, 0.2356619118451075)]

In [50]:
max_movies_recommendations = 10

print("*** Suggested movies for you ***\n")
for idx, cosine in sorted_similar_movies[0:max_movies_recommendations]:
    movie = movies_data[movies_data["index"] == idx]["title"].values[0]
    print(movie)

*** Suggested movies for you ***

Iron Man
Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
Captain America: The Winter Soldier
Ant-Man
X-Men
Made


## Movie Recommendation System Resume

In [57]:
movie_name = "lord of the rings"

all_movies_names_list = movies_data["title"].tolist()

find_close_match = difflib.get_close_matches(movie_name, all_movies_names_list)

close_match = find_close_match[0] if len(find_close_match) > 0 else "N/A"

movie_index = movies_data[movies_data["title"] == close_match]["index"].values[0]

similarity_score = list(enumerate(similarity[movie_index]))

sorted_similar_movies = sorted(similarity_score, key= lambda x: x[1], reverse=True)

max_movies_recommendations = 10

#print(find_close_match)

print("*** Suggested movies for you ***\n")
for idx, cosine in sorted_similar_movies[0:max_movies_recommendations]:
    movie = movies_data[movies_data["index"] == idx]["title"].values[0]
    print(movie)

['Wrath of the Titans', 'Clash of the Titans', 'George of the Jungle']
*** Suggested movies for you ***

Wrath of the Titans
Clash of the Titans
Hands of Stone
The Legend of Hercules
Guten Tag, Ramón
Joy
Percy Jackson & the Olympians: The Lightning Thief
Deliver Us from Evil
Domino
Schindler's List


In [59]:
# Guardamos la matriz de similitudes para uso posterior con API

np.save('movies_similarity_matrix.npy', similarity)